In [ ]:
# Magic methods that automatically update changes to imported python functions

%load_ext autoreload
%autoreload 2

In [1]:
year = 2019
url = f'https://api.census.gov/data/{year}/abscs?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F&for=us:*'

## Questions to Answer

1. Question 1 - Visual Type A : Time in business vs Wealth
2. Question 2 - Visual Type A : Size of business vs Wealth
3. Question 3 - Visual Type B : Employee Size vs Wealth

4. Question 4 - Visual Type A : Demographic of Employees
5. Question 5 - Visual Type A : Demographic of Employees
6. Question 6 - Visual Type B : Demographic of Employees

7. Question 7 - Visual Type A : Wealth by Business Type
8. Question 8 - Visual Type A : Wealth by Location
9. Question 9 - Visual Type B : Mystery Question


In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as pl
import seaborn as sns

YearinBusinessDF = pd.read_csv('csv\ABSCS2018.AB1800CSA02-2023-04-20T164944.csv')
ReceiptSizeOfFirmDF = pd.read_csv('csv\ABSCS2018.AB1800CSA03-2023-04-20T164951.csv')
EmploymentSizeOfFirmDF = pd.read_csv('csv\ABSCS2018.AB1800CSA04-2023-04-20T164954.csv')

listOfDF = [YearinBusinessDF, ReceiptSizeOfFirmDF, EmploymentSizeOfFirmDF]

response = requests.get(url)
response

<Response [200]>

In [3]:
# The easy way to read in API data

data = response.json()

col = data[0]

df = pd.DataFrame(data[1:], columns=col)

colToDrop = ['GEO_ID', 'NAME', f'NAICS{year - 1}', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'PAYANN_F',
 'FIRMPDEMP_S', 'FIRMPDEMP_S_F', 'RCPPDEMP_S', 'RCPPDEMP_S_F', 'EMP_S', 'EMP_S_F', 'PAYANN_S',
 'PAYANN_S_F', 'us', 'YEAR', 'FIRMPDEMP_F', 'RCPPDEMP_F', 'EMP_F', 'VET_GROUP', 'EMPSZFI']

df = df.drop(columns= colToDrop, axis = 1).convert_dtypes().drop_duplicates()

# list(df.columns)
df.head()
# df.shape

KeyError: "['NAICS2018'] not found in axis"

In [7]:
# Distinct column values

# df[['Column Name']].drop_duplicates()

,NAICS2017_LABEL
0,Total for all sectors
1144,"Agriculture, forestry, fishing and hunting"
1450,"Mining, quarrying, and oil and gas extraction"
2056,Manufacturing
2178,Utilities
2750,Construction
4754,Wholesale trade
6048,Retail trade
6844,Transportation and warehousing
7624,Information


Columns to keep from YearinBusinessDF:

 'Geographic Area Name (NAME)',
 '2017 NAICS code (NAICS2017)',
 'Meaning of NAICS code (NAICS2017_LABEL)',
 'Meaning of Sex code (SEX_LABEL)',
 'Meaning of Ethnicity code (ETH_GROUP_LABEL)',
 'Meaning of Race code (RACE_GROUP_LABEL)',
 'Meaning of Veteran code (VET_GROUP_LABEL)',
 'Meaning of Years in business code (YIBSZFI_LABEL)',
 'Number of employer firms (FIRMPDEMP)',
 'Sales, value of shipments, or revenue of employer firms ($1,000) (RCPPDEMP)',
 'Number of employees (EMP)',
 'Annual payroll ($1,000) (PAYANN)'

Columns to remove:

 'Year (YEAR)',
 'Relative standard error of employer firms (%) (FIRMPDEMP_S)',
 'Relative standard error of sales, value of shipments, or revenue of employer firms (%) (RCPPDEMP_S)',
 'Relative standard error of number of employees (%) (EMP_S)',
 'Relative standard error of annual payroll (%) (PAYANN_S)'
Column to merge from EmploymentSizeOfFirmDF:

- 'Meaning of Employment size of firms code (EMPSZFI_LABEL)'

 Column to merge from ReceiptSizeOfFirmDF:

- 'Meaning of Sales, value of shipments, or revenue size of firms code (RCPSZFI_LABEL)'

## Cleaning

- Drop columns we aren't planning to use
- Rename Columns we are planning to use
- Merge

###Visualization 
1. years in Business vs count of Business

In [5]:
YearinBusinessDF.columns

Index(['Geographic Area Name (NAME)', '2017 NAICS code (NAICS2017)',
       'Meaning of NAICS code (NAICS2017_LABEL)',
       'Meaning of Sex code (SEX_LABEL)',
       'Meaning of Ethnicity code (ETH_GROUP_LABEL)',
       'Meaning of Race code (RACE_GROUP_LABEL)',
       'Meaning of Veteran code (VET_GROUP_LABEL)',
       'Meaning of Years in business code (YIBSZFI_LABEL)', 'Year (YEAR)',
       'Number of employer firms (FIRMPDEMP)',
       'Sales, value of shipments, or revenue of employer firms ($1,000) (RCPPDEMP)',
       'Number of employees (EMP)', 'Annual payroll ($1,000) (PAYANN)',
       'Relative standard error of employer firms (%) (FIRMPDEMP_S)',
       'Relative standard error of sales, value of shipments, or revenue of employer firms (%) (RCPPDEMP_S)',
       'Relative standard error of number of employees (%) (EMP_S)',
       'Relative standard error of annual payroll (%) (PAYANN_S)'],
      dtype='object')

In [6]:
yb_df1 = YearinBusinessDF

In [ ]:
yb_df1= yb_df1.loc[1:, ['YIBSZFI_LABEL']]
yb_df1['YIBSZFI_LABEL'] = yb_df1['YIBSZFI_LABEL'].replace({
    'Firms with less than 2 years in business': '0 to 2', 
    'Firms with 2 to 3 years in business': '2-3', 
    'Firms with 4 to 5 years in business': '4-5',
    'Firms with 6 to 10 years in business' : '6-10',
    'Firms with 11 to 15 years in business' : '11-15',
    'Firms with 16 or more years in business' : '16 plus',
})

In [ ]:
# GRaph 1
#### Filter the DataFrame to exclude rows with YIBSZFI_LABEL equal to 'Meaning of years in business code'
yb_df = yb_df1.loc[yb_df1['YIBSZFI_LABEL'] != 'Meaning of Years in business code']

# Plot the data using the filtered DataFrame
plt.bar(yb_df['YIBSZFI_LABEL'].value_counts().index, yb_df['YIBSZFI_LABEL'].value_counts().values)
plt.xlabel('Years of the Business')
plt.ylabel('Count of the business')
plt.title('Business years and counts')
plt.show()

In [ ]:
yb_df2 = YearinBusinessDF.loc[YearinBusinessDF['RACE_GROUP_LABEL']
                              != 'Meaning of Race code']
yb_df2[['RACE_GROUP_LABEL']].drop_duplicates()


In [ ]:
replacement_dict = {
    'American Indian and Alaska Native': 'Native American ', 
    'Native Hawaiian and Other Pacific Islander': 'Islander', 
    'Equally minority/nonminority': 'Minority',
    'Black or African American': 'African American',
    'Total': 'Unclassifiable'
}

yb_df2['RACE_GROUP_LABEL'] = yb_df2['RACE_GROUP_LABEL'].replace(replacement_dict)

In [ ]:
#Tree map for SEX_LABEL 
#### male and female proportion on business in USA(exclude total from values)
import plotly.express as px
# count the number of value in the SEX_LABEL column, excluding the "Total" value
grouped_df = yb_df2[yb_df2['SEX_LABEL'] != 'Total']['SEX_LABEL'].value_counts().reset_index(name='Count')
grouped_df.rename(columns={'index': 'Sex'}, inplace=True)

# create treemap with the grouped data
fig = px.treemap(grouped_df, path=['Sex'], values='Count')

# show the plot
fig.show()